# **Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia**
---

In [1]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [2]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    

In [3]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [4]:
response = requests.get(static_url)

In [7]:
response.status_code

200

In [13]:
soup = BeautifulSoup(response.content, "html.parser")

In [15]:
soup.title.get_text()

'List of Falcon 9 and Falcon Heavy launches - Wikipedia'

In [46]:
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

---

In [16]:
html_tables = soup.find_all('table')

In [18]:
first_launch_table = html_tables[2]
#print(first_launch_table)

In [32]:
column_names = []

columns = first_launch_table.find_all('th')
for column in columns:
    column_name = column.get_text(strip=True)
    if column_name is not None and len(column_name) > 1:
        column_names.append(column.get_text(strip=True))

In [33]:
column_names

['Flight No.',
 'Date andtime (UTC)',
 'Version,Booster[b]',
 'Launch site',
 'Payload[c]',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launchoutcome',
 'Boosterlanding']

In [38]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date andtime (UTC)']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload[c]'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launchoutcome'] = []
# Added some new columns
launch_dict['Version,Booster[b]']=[]
launch_dict['Boosterlanding']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [41]:
 extracted_row = 0
        
# Funciones auxiliares (debes definir estas funciones para tu caso)
def date_time(cell):
    return cell.get_text(strip=True).split(' ')

def booster_version(cell):
    return cell.get_text(strip=True)

def get_mass(cell):
    return cell.get_text(strip=True).split(' ')[0]

def landing_status(cell):
    return cell.get_text(strip=True)

# Extraer cada fila de la tabla
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Obtener las filas de la tabla
    for rows in table.find_all("tr"):
        # Verificar si el primer encabezado de la fila es un número correspondiente a un lanzamiento
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
            else:
                flag = False
        else:
            flag = False
        
        # Obtener los elementos de la fila
        row = rows.find_all('td')
        
        # Si es un número, guardar las celdas en un diccionario
        if flag:
            extracted_row += 1
            # Número de vuelo
            launch_dict['Flight No.'].append(flight_number)
            
            # Fecha y hora
            datatimelist = date_time(row[0])
            launch_dict['Date'].append(datatimelist[0].strip(','))
            launch_dict['Time'].append(datatimelist[1])
            
            # Versión del booster
            bv = booster_version(row[1])
            if not bv:
                bv = row[1].a.string if row[1].a else None
            launch_dict['Version,Booster[b]'].append(bv)
            
            # Sitio de lanzamiento
            launch_site = row[2].a.string if row[2].a else None
            launch_dict['Launch site'].append(launch_site)
            
            # Payload
            payload = row[3].a.string if row[3].a else None
            launch_dict['Payload[c]'].append(payload)
            
            # Masa del payload
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            # Órbita
            orbit = row[5].a.string if row[5].a else None
            launch_dict['Orbit'].append(orbit)
            
            # Cliente
            customer = row[6].a.string if row[6].a else None
            launch_dict['Customer'].append(customer)
            
            # Resultado del lanzamiento
            launch_outcome = list(row[7].strings)[0] if row[7].strings else None
            launch_dict['Launchoutcome'].append(launch_outcome)
            
            # Aterrizaje del booster
            booster_landing = landing_status(row[8])
            launch_dict['Boosterlanding'].append(booster_landing)

# Imprimir el diccionario de lanzamiento
print(launch_dict)

{'Flight No.': ['1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121'], 'Version,Booster[b]': ['F9 v1.0[7]B0003.1[8]', 'F9 v1.0[7]B0004.1[8]', 'F9 v1.0[7]B0005.1[8]', 'F9 v1.0[7]B0006.1[8]', 'F9 v1.0[7]B0007.1[8]', 'F9 v1.1[7]B1003[8]', 'F9 v1.1B1004', 'F9 v1.1', 'F9 v1.1', 'F9 v1.1', 'F9 v1.1', 'F9 v1.1B1011[

In [42]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [44]:
df.head()

,Flight No.,"Version,Booster[b]",Launch site,Payload[c],Payload mass,Orbit,Customer,Launchoutcome,Boosterlanding,Date,Time
0,1,F9 v1.0[7]B0003.1[8],CCAFS,Dragon Spacecraft Qualification Unit,,LEO,SpaceX,Success\n,Failure[9][10](parachute),4,June
1,1,F9 v1.0[7]B0004.1[8],CCAFS,Dragon,,LEO,NASA,Success,Failure[9][14](parachute),4,June
2,2,F9 v1.0[7]B0005.1[8],CCAFS,Dragon,525 kg,LEO,NASA,Success,No attempt,8,December
3,3,F9 v1.0[7]B0006.1[8],CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,No attempt,22,May
4,4,F9 v1.0[7]B0007.1[8],CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,No attempt,8,October


In [43]:
df.to_csv('spacex_web_scraped.csv', index=False)